*Stanislav Borysov [stabo@dtu.dk], DTU Management*
# Advanced Business Analytics

## Deep Learning - Part 1 - Introduction to Keras, Convolutional Neural Networks (CNNs)

Deep learning recently caused a lot of hype. Some people call it AI, some just sophisticated "curve fitting". However, the fact is that it pushed state-of-the-art performance in multiple fields, especially, in computer vision, natural language processing, and speech recognition. It also started gaining popularity in business analytics where the data are often high dimensional and have a rich structure, like spatiotemporal data from smart mobility (https://www.kaggle.com/c/nyc-taxi-trip-duration/kernels). Today, we'll focus on multi-layer neural networks in Keras — one of the most popular deep learning frameworks designed for both research and production.

Being quite general and flexible, an NN has become a very popular and powerful model with a wide range of applications. One of its main strengths is the ability to process information hierarchically, automatically capturing new levels of abstraction in the data, effectively dealing with the curse of dimensionality. This problem arises because, with the growth of dimensionality (i.e., the number of features) of the data, the volume of a unit sphere in this space growths exponentially. As a consequence, to explore this volume and provide reasonable statistical estimations, a model requires exponentially more data samples. One of the known approaches to tackle this problem is dimensionality reduction, which assumes that there are only a few important dimensions, which can be represented as (often nonlinear) combinations of the initial ones. An NN is believed to do it automatically, where each following layer learns new low-dimensional representations of the data.

For example, consider a document topic classification problem given scanned handwritten text as an input. It would be almost impossible to perform this semantic analysis directly on the raw pixel intensities. Instead, an NN detects strokes and curves first, then tries to identify letters, which in turn constitute words. This can be achieved thanks to the layered structure that theoretically can learn any complex mapping. We stress the word “theoretically” because it appears not so trivial to do it in practice. The main challenge is the training of an NN: Fitting its parameters essentially boils down to finding a minimum of a nonconvex function in a high (thousand- or even million-) dimensional space. Therefore, plain fully connected NNs are rarely used alone. Many different hybrid architectures of NNs tailored for different purposes have been proposed. A Convolutional Neural Network (CNN) and a Recurrent Neural Network (RNN) are the two most fundamental prototypes which can be found in almost all modern deep learning models.

<img src="ann.png">

### 1. Multilayer Neural Networks in Keras

Keras is a useful package that provides a high-level interface to more advanced deep learning frameworks such as TensorFlow. It makes the life of a deep learning researcher much easier. 

https://keras.io/

"Keras is a high-level neural networks API, written in Python and capable of running on top of TensorFlow, CNTK, or Theano. It was developed with a focus on enabling fast experimentation. Being able to go from idea to result with the least possible delay is key to doing good research.

Use Keras if you need a deep learning library that:

- Allows for easy and fast prototyping (through user-friendliness, modularity, and extensibility).
- Supports both convolutional networks and recurrent networks, as well as combinations of the two.
- Runs seamlessly on CPU and GPU."

It has two ways of building models: Sequential and Functional APIs.

#### 1.1 Sequential API

The Sequential API helps to start with something really simple. But let's import Keras and prepare the data first.

In [ ]:
import keras

In this lecture, we will use the MNIST database of handwritten digits - the most popular toy example in deep learning. It has a training set of 60,000 examples and a test set of 10,000 examples of 28x28 pixels each. The digits have been size-normalized and centered in a fixed-size image.

In [ ]:
from keras.datasets import mnist

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# input image dimensions
img_rows, img_cols = 28, 28
# number of labels
num_classes = 10

# conver images to 1D vectors
x_train = x_train.reshape(x_train.shape[0], img_rows * img_cols)
x_test = x_test.reshape(x_test.shape[0], img_rows * img_cols)

# correct data type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalize picxel values from [0, 255] to [0, 1]
x_train /= 255
x_test /= 255

Let's visualize some data samples first.

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline


def plot_digits(samples, n=20):
    ind = np.arange(len(samples))
    np.random.shuffle(ind)
    ind = ind[:n**2]
    figure = np.zeros((img_rows * n, img_cols * n))
    grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
    grid_y = norm.ppf(np.linspace(0.05, 0.95, n))
    for i, yi in enumerate(grid_x):
        for j, xi in enumerate(grid_y):
            indx = i * n + j
            sample = samples[indx]
            digit = sample.reshape(img_rows, img_cols)
            figure[i * img_rows: (i + 1) * img_rows,
                   j * img_cols: (j + 1) * img_cols] = digit
    plt.figure(figsize=(15, 15))
    plt.imshow(figure, cmap='Greys_r')
    plt.show()
    

plot_digits(x_train)

Although it is a relatively small dataset, training neural nets on it requires a lot of resources! So let's limit our data by 10,000 training and 5000 test samples, which is more suitable for a regular laptop.

In [ ]:
n_train, n_test = 10000, 5000
x_train, y_train = x_train[:n_train], y_train[:n_train]
x_test, y_test = x_test[:n_test], y_test[:n_test]

We are going to solve the classification task: given an image, we need to predict which number is depicted on it. The labels in ```y_train``` and ```y_test``` are integers from 0 to 9. To use them in the ML, as usual, we need to convert them to the one-hot representation. For example, 0 becomes [1, 0, 0, 0, 0, 0, 0, 0, 0, 0], 2 becomes [0, 0, 1, 0, 0, 0, 0, 0, 0, 0], etc. Fortunately, Keras has a lot of convenient built-in functons.

In [ ]:
num_classes = 10

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Great! Now we're ready to build our first "deep" neural network model using the Keras Sequential API.

A sequential model is specified using the ```Sequential()``` class.

In [ ]:
from keras.models import Sequential

model = Sequential()

Let's add two densely connected layers to it consisting of 64 neurons each with ```tanh``` (hyperbolic tangent) nonlinearity. ```tanh``` is very similar to the sigmoid function used in logistic regression but produces values from -1 to 1 instead of [0, 1]. Another common choice is ```relu``` (Rectified Linear Unit) which is simply ReLU(x)=max(0,x).

<img src="activations.png">

In [ ]:
from keras.layers import Dense

act_f = 'tanh' # There are a lot of them! See https://keras.io/activations/
hidden_neurons_n = 64

model.add(Dense(hidden_neurons_n, activation=act_f, input_shape=(img_rows * img_cols,))) # if no activation is specified, the layer becomes linear
model.add(Dense(hidden_neurons_n, activation=act_f))

The (final) output layer has ```softmax``` activation,

\begin{equation*}
\mathrm{softmax}(x_i) = \frac{\exp{x_i}}{\sum_i\exp{x_i}}.
\end{equation*}

The ```softmax``` function is a common choice for classification problems. Its output is normalized to 1 and represents probabilities for each class. For example, the output [0.1, 0.5, 0.1, 0.1, 0.1, 0.02, 0.02, 0.02, 0.02, 0.02] means that the NN thinks that (most probably) it is the digit "1".

In [ ]:
model.add(Dense(num_classes, activation='softmax'))

Let's print a summary of our model.

In [ ]:
model.summary()

Even such a small NN with 2 hidden layers of 64 neurons each already has 55,050 parameters to optimize! You can now imagine how hard was to train deep NN 20 years ago...

OK, the final step before actual training is to compile our model to build a TensorFlow computational graph. We also need to specify an optimizer and loss function to use. Most optimizers for NN are based on the gradient descent algorithm, which helps to adjust parameters in order to minimize the difference between the NN predictions and actual ground truth. For regression tasks, the mean squared error is commonly used. For binary classification, we already used binary loss, the same as in the logistic regression. Cross-entropy loss is a generalization of the binary loss for multi-class problems. Note, that, in the case of NNs, we don't need to use "one-versus-all" or any other tricks, so the only one model is sufficient.

In [ ]:
from keras.optimizers import RMSprop # there are a lot of them! See https://keras.io/optimizers/

optimizer = RMSprop() # you can also specify parameters of the otimizer, e.g. learning rate: RMSprop(lr=0.001)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])

Huh, a lot of preparations... The modern deep learning frameworks require a lot of work to be done, however, it pays off with fast training which was almost unimaginable a few years ago.

The model can be trained using the ```fit()``` function. Some parameters to specify:
- ```batch_size``` - Calculating gradients over all the data is usually time-consuming. That's why the training data is usually divided into chunks called "mini-batches". Although the gradients, in this case, become approximate, on practice, it even helps to get better solutions, for example, by escaping from local minima of the loss function.
- ```epochs``` - Number of epochs to be used. One epoch corresponds to one pass through the whole train data. If ```batch_size=len(x_train)```, then one batch corresponds to one epoch.
- ```validation_data``` - The data to be used not for the test purposes on-the-fly (to detect overfitting).

Let's go!

In [ ]:
batch_size = 128
epochs = 10

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1, # show training process
                    validation_data=(x_test, y_test))

Let's check the test accuracy one more time

In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Congrats, ~92% accuracy is not bad. Let's check if we can get better results with a larger number of epochs, different batch sizes, more/larger layers, different activation functions [https://keras.io/activations/], optimizers and their parameters like learning rate [https://keras.io/optimizers/] (These all are hyperparameters of the model!). Try to build and train a new model yourself now.

In [ ]:
my_model = Sequential()
#...

#### 1.2 Functional API

Let also familiarize ourselves with the Functional API. The Keras Functional API is the way to go for defining complex models, such as multi-output models, directed acyclic graphs, or models with shared layers. It's not very different from the Sequential but is more flexible. It requires to specify inputs and variables (tensors) passing to the layers explicitly.

In [ ]:
from keras.layers import Input
from keras.models import Model

# This returns a tensor
inputs = Input(shape=(img_rows * img_cols,))

# a layer instance is callable on a tensor, and returns a tensor
x = Dense(hidden_neurons_n, activation=act_f)(inputs)
x = Dense(hidden_neurons_n, activation=act_f)(x)
predictions = Dense(num_classes, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=inputs, outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))  # starts training

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

*Note that the results might be a bit different because parameters of the NN (weights and biases) are initialized randomly (by default) every time you specify a new model.*

Implement your model using the Functional API now.

In [ ]:
# ...

### 2. Convolutional Neural Networks (CNNs)

A dense Neural Network is a very powerful model capable of approximating arbitrary nonlinear functions. At the same time, due to a large number of parameters, dense NNs are prone to overfitting. The enormous amount of data allowed for training them more efficiently nowadays. However, the real revolution began when special architectures of NNs, like CNNs and RNNs, came into play. Of course, the MNIST classification problem is relatively easy, so do not expect much gain from using a CNN instead of a plain fully connected (dense) NN.

A CNN architecture is believed to resemble a visual system of a brain. It consists of several convolutional-pooling layer pairs followed by a fully connected (dense) network. Neurons in a convolutional layer, which share their weights, scan through the input and produce multiple outputs of the same size as the input, also known as feature maps. The layer is called convolutional because it literarily performs convolutions of the input using adaptive kernels represented by a dense NN with one layer. A pooling layer downsamples these feature maps, using, for example, the ```max``` function over small contiguous regions, effectively reducing their dimensionality. These layers preserve spatial correlations and are capable of capturing the beforementioned hierarchical features, such as strokes, basic geometric shapes and so on. These layers are followed by a fully connected NN with a small number of hidden layers which is now able to solve the initially complex task (e.g. classification of cats versus not-cats).

<img src="cnn.png">

In Keras, the two special CNN layers are implemented using ```Conv2D``` and ```MaxPooling2D``` layers. To convert tensor to a 1D array, one can use ```Flatten``` layer.

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten

As CNNs are primarily used for image processing tasks, these layers designed to use 3 channels (RGB) by default. As the MNIST data is grayscale, i.e. has only 1 channel, we need to specify this explicitly by reshaping the data.

In [ ]:
from keras import backend as K

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

Let's build our first CNN...

In [ ]:
batch_size = 128
epochs = 10

kernels_n = 16
hidden_neurons_dense_n = 64
act_f = 'tanh'
kernel_size = (3, 3)
pool_size = (2, 2)

optimizer = RMSprop() # you can also specify parameters of the otimizer, e.g. learning rate: RMSprop(lr=0.001)


model = Sequential()
model.add(Conv2D(kernels_n, kernel_size=kernel_size,
                 activation=act_f,
                 input_shape=input_shape))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Flatten())
model.add(Dense(hidden_neurons_dense_n, activation=act_f))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.summary()

...and train it (Notice the number of the parameters!)

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Congrats, ~94% accuracy is not bad. Let's check if we can get a better result with a larger number of epochs, different batch size, more convolutional+pooling layers, more/larger dense layers, different activation functions [https://keras.io/activations/], optimizers and their parameters like learning rate [https://keras.io/optimizers/] (These all are hyperparameters of the model!). Try to build and train a new model yourself now using first Sequential and then Functional API.

In [ ]:
# Sequential API

my_cnn = Sequential()
# ...

In [ ]:
# Functional API

inputs = Input(shape=input_shape)
# ...